# Personal Finance

Aswin van Woudenberg (https://www.aswinvanwoudenberg.com | https://github.com/afvanwoudenberg)

## Importing dependencies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets as widgets
import numpy as np

import calendar
from datetime import datetime, timedelta

import os

from babel.numbers import format_currency

from ipywidgets import interact

## Reading CAMT.053 files

The following constant defines where we will read the `CAMT.053` files from.

In [ ]:
IMPORT_PATH = "import"

In [ ]:
groupheaders = pd.DataFrame()
statements = pd.DataFrame()
balances = pd.DataFrame()
entries = pd.DataFrame()

for file in os.listdir(IMPORT_PATH):
    # groupheaders
    df = pd.read_xml(os.path.join(IMPORT_PATH, file), stylesheet="groupheader.xsl")
    df.insert(0, "FileName", file)
    groupheaders = pd.concat([groupheaders, df], ignore_index=True)

    # statements
    df = pd.read_xml(os.path.join(IMPORT_PATH, file), stylesheet="statement.xsl")
    df.insert(0, "FileName", file)
    statements = pd.concat([statements, df], ignore_index=True)

    # balances
    df = pd.read_xml(os.path.join(IMPORT_PATH, file), stylesheet="balance.xsl")
    df.insert(0, "FileName", file)
    balances = pd.concat([balances, df], ignore_index=True)

    df = pd.read_xml(os.path.join(IMPORT_PATH, file), stylesheet="entry.xsl")
    df.insert(0, "FileName", file)
    entries = pd.concat([entries, df], ignore_index=True)

groupheaders['CreationDateTime'] = pd.to_datetime(groupheaders.CreationDateTime)
groupheaders.sort_values('CreationDateTime', ignore_index=True, inplace=True)

statements['CreationDateTime'] = pd.to_datetime(statements.CreationDateTime)
statements.sort_values('CreationDateTime', ignore_index=True, inplace=True)

balances['Date'] = pd.to_datetime(balances.Date)
balances.sort_values('Date', ignore_index=True, inplace=True)

entries['BookingDate'] = pd.to_datetime(entries.BookingDate)
entries['ValueDate'] = pd.to_datetime(entries.ValueDate)
entries.sort_values('BookingDate', ignore_index=True, inplace=True)

In [ ]:
groupheaders.info()

In [ ]:
statements.info()

In [ ]:
balances.info()

In [ ]:
entries.info()

## Account balance over time

In [ ]:
@interact(iban=widgets.Dropdown(options=statements.IBAN.unique(), description='Account:'))
def plot_account_balance(iban):
    statement_balances = pd.merge(left=balances, right=statements, left_on=['FileName','Identification'], right_on=['FileName','Identification']).query('Code=="CLBD" & IBAN==@iban')
    statement_balances.loc[statement_balances['CreditDebitIndicator'] == 'DBIT', 'Amount'] *= -1

    xticks = [(y, m) for y, m in statement_balances.groupby([statement_balances.Date.dt.year, statement_balances.Date.dt.month]).size().index]
    xtick_labels = ["{}, {}".format(y, m) for y, _, m in statement_balances.groupby([statement_balances.Date.dt.year, statement_balances.Date.dt.month, statement_balances.Date.dt.month_name()]).size().index]

    ax = (statement_balances
        .groupby([statement_balances.Date.dt.year, statement_balances.Date.dt.month])['Amount'].mean()
        .plot(kind='line', figsize=(14,5))
    )
    ax.set_xticks(np.arange(0,len(xticks),1))
    ax.set_xticklabels(xtick_labels, rotation = 90)
    ax.set_xlabel(None)
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("\u20ac%d"))
    
    plt.plot()

## Income vs. expenses over time

In [ ]:
dates = pd.date_range(entries.ValueDate.min(), entries.ValueDate.max() + timedelta(days=31), freq='M')

In [ ]:
@interact(
    iban=widgets.Dropdown(options=statements.IBAN.unique(), description='Account:', layout={'width': '500px'}), 
    date_range=widgets.SelectionRangeSlider(
        options=[(date.strftime(' %b %Y '), date) for date in dates],
        index=(0, len(dates)-1),
        description='Dates:',
        layout={'width': '500px'}
    )
)
def plot_income_expenses(iban, date_range):
    start_date = datetime(date_range[0].year, date_range[0].month, 1)
    end_date = date_range[1]
    transactions = pd.merge(left=statements, right=entries, left_on=['FileName', 'Identification'], right_on=['FileName', 'Identification']).query('IBAN == @iban & ValueDate >= @start_date & ValueDate <= @end_date')
    
    xticks = ["{}, {}".format(y, m) for y, _, m in transactions.groupby([transactions.ValueDate.dt.year, transactions.ValueDate.dt.month, transactions.ValueDate.dt.month_name()]).size().index]
    
    ax = (transactions
        .groupby([transactions.ValueDate.dt.year, transactions.ValueDate.dt.month, 'CreditDebitIndicator'])['Amount'].sum().unstack('CreditDebitIndicator')
        .plot(kind='bar', figsize=(14,5), color=['tab:blue', 'tab:orange'])
    )
    ax.legend(['Income', 'Expenses'])
    ax.set_xticklabels(xticks)
    ax.set_xlabel(None)
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("\u20ac%d"))

    # Calculate average values
    averages = transactions.groupby([transactions.ValueDate.dt.year, transactions.ValueDate.dt.month, 'CreditDebitIndicator'])['Amount'].sum().unstack('CreditDebitIndicator').mean()
    
    # Plot average lines
    ax.axhline(y=averages.get('CRDT', 0), linestyle='--', color='tab:blue')
    ax.axhline(y=averages.get('DBIT', 0), linestyle='--', color='tab:orange')
    
    plt.plot()

## Heatmap of income and expenses

In [ ]:
@interact(iban=widgets.Dropdown(options=statements.IBAN.unique(), description='Account:'))
def plot_heatmap_income_expenses(iban):
    transactions = pd.merge(left=statements, right=entries, left_on=['FileName', 'Identification'], right_on=['FileName', 'Identification']).query('IBAN == @iban')
    transactions.loc[transactions['CreditDebitIndicator'] == 'DBIT', 'Amount'] *= -1
    df = (transactions.groupby([transactions.ValueDate.dt.year, transactions.ValueDate.dt.month])['Amount'].sum().unstack(1)
        .rename_axis('Year').rename_axis('Month', axis='columns')
        .reindex(list(range(1,13)), axis='columns', fill_value=0)
        .rename(columns=lambda x: list(calendar.month_name)[x]))
    v = max(df.max(axis=None), df.min(axis=None))
    display(df
        .style.format(na_rep=0, precision=2).background_gradient(cmap='RdBu', vmin=-v, vmax=v)
        .applymap(lambda x: 'background-color: white; color: white;' if pd.isnull(x) or x==0 else '')
    )

## Categorizing transactions

In [ ]:
transactions = pd.merge(left=statements, right=entries, left_on=['FileName', 'Identification'], right_on=['FileName', 'Identification'])

Add and update categories and corresponding keywords below in order to categorize your transactions.

In [ ]:
categories = {
    "Income": ["Salary", ...],
    "Giving": ["Gift", "WWF", ...],
    "Saving": ["Savings", "Retirement", ...],
    "Food": ["Albert Heijn", "Jumbo", "Lidl", "Starbucks", "Restaurant", ...],
    "Utilities": ["Vitens", "KPN", "Vattenfall", "Internet Services", ...],
    "Housing": ["Mortage", ...],
    "Transportation": ["NS", "OV-Chipkaart", "Uber", "Lyft", ...],
    "Health": ["Infomedics", ...],
    "Insurance": ["AEGON", "Centraal beheer", ...],
    "Trips and Entertainment": ["Booking.com", "AirBnB", ...],
    "Personal Spending": ["Hairsalon", "Shoes", ...],
    "Miscellaneous": ["ATM", ...]
}

In [ ]:
def categorize_transaction(row):
    columns = ['AdditionalEntryInformation', 'DetailsRemittanceInformationUnstructured']
    for (category, keywords) in categories.items():
        for column in columns:
            if isinstance(row[column], str) and any([row[column].lower().find(kw.lower()) >= 0 for kw in keywords]):
                return category
    return None

In [ ]:
transactions['Category'] = transactions.apply(categorize_transaction, axis=1)

## Inspecting categories

In [ ]:
@interact(iban=widgets.Dropdown(options=statements.IBAN.unique(), description='Account:'), category=widgets.Dropdown(options=list(categories.keys()) + ['Uncategorized'], description='Category:'))
def show_transactions(iban, category):
    columns = ['IBAN', 'Amount','CreditDebitIndicator', 'BookingDate', 'ValueDate', 'Issuer', 'AdditionalEntryInformation', 'DetailsAmount', 'DetailsCurrency',
       'DetailsCreditorName', 'DetailsCreditorIBAN', 'DetailsRemittanceInformationUnstructured', 'DetailsDebtorName', 'DetailsDebtorIBAN', 'DetailsCreditorCountry']
    if category == "Uncategorized":
        display(transactions.query('IBAN == @iban & Category.isnull()')[columns])
        # transactions.query('IBAN == @iban & Category.isnull()')[columns].to_csv('uncategorized.csv')
    else:
        display(transactions.query('IBAN == @iban & Category == @category')[columns])

## Plotting the expenses per category

In [ ]:
@interact(
    iban=widgets.Dropdown(options=statements.IBAN.unique(), description='Account:', layout={'width': '500px'}), 
    date_range=widgets.SelectionRangeSlider(
        options=[(date.strftime(' %b %Y '), date) for date in dates],
        index=(0, len(dates)-1),
        description='Dates:',
        layout={'width': '500px'}
    )
)
def plot_expenses_by_category(iban, date_range):
    start_date = datetime(date_range[0].year, date_range[0].month, 1)
    end_date = date_range[1]
    df = transactions.query('IBAN == @iban & ValueDate >= @start_date & ValueDate <= @end_date & Category != "Income"').groupby('Category')['Amount'].sum()
    df.to_csv('exp_cat.csv')
    total = df.sum()
    ax = df.plot(kind='pie', startangle=90, figsize=(10,10), legend=False, autopct=lambda v: '{:.1f}%\n\u20ac{:.2f}'.format(v, total*v/100))
    ax.set_title("Expenses")
    ax.axis('off')
    ax.add_artist(plt.Circle(xy=(0,0), radius=.75, facecolor='white'))
    plt.show()